In [19]:
from langchain_core.tools import tool

#Tool annotation identifies a function as a tool automatically
@tool
def find_sum(x:int, y:int) -> int :
    #The docstring comment describes the capabilities of the function
    #It is used by the agent to discover the function's inputs, outputs and capabilities
    """
    This function is used to add two numbers and return their sum.
    It takes two integers as inputs and returns an integer as output.
    """
    return x + y

@tool
def find_product(x:int, y:int) -> int :
    """
    This function is used to multiply two numbers and return their product.
    It takes two integers as inputs and returns an integer as ouput.
    """
    return x * y

In [21]:
from langchain_aws import ChatBedrock
import boto3

session = boto3.Session(profile_name="default")
bedrock_client = session.client('bedrock-runtime')

model_id = 'anthropic.claude-3-haiku-20240307-v1:0'
# model_id = "meta.llama3-8b-instruct-v1:0"
model_kwargs =  { 
        "max_tokens": 512,
        "temperature": 0,
        "top_p": 1,
}

# LangChain class for chat
model = ChatBedrock(
    client=bedrock_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

#Test the model
# response = model.invoke("Hello, how are you?")
# print(response.content)

Hello! As an AI language model, I don't have feelings or a physical form, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?


In [26]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create list of tools available to the agent
agent_tools=[find_sum, find_product]

#System prompt
system_prompt = SystemMessage(
    """You are a Math genius who can solve math problems. Solve the
    problems provided by the user, by using only tools available. 
    Do not solve the problem yourself"""
)

agent_graph=create_react_agent(
    model=model, 
    state_modifier=system_prompt,
    tools=agent_tools)

In [27]:
#Example 1
inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())

Agent returned : The sum of 2 and 3 is 5. 

Step by Step execution : 
================================ Human Message =================================

what is the sum of 2 and 3 ?
================================== Ai Message ==================================
Tool Calls:
  find_sum (toolu_bdrk_01FCq6y5D1N5j8uZQDDM77HB)
 Call ID: toolu_bdrk_01FCq6y5D1N5j8uZQDDM77HB
  Args:
    x: 2
    y: 3
================================= Tool Message =================================
Name: find_sum

5
================================== Ai Message ==================================

The sum of 2 and 3 is 5.


In [32]:
#Example 2
inputs = {"messages":[("user","What is 3 multipled by 2 and 5 + 2 ?")]}

result = agent_graph.invoke(inputs)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())

Agent returned : Okay, so 5 + 2 is 7.

Therefore, the final result is 6 (3 multiplied by 2) plus 7 (5 + 2), which is 13. 

Step by Step execution : 
================================ Human Message =================================

What is 3 multipled by 2 and 5 + 2 ?
================================== Ai Message ==================================

Okay, let's solve this step-by-step using the available tools:
Tool Calls:
  find_product (toolu_bdrk_01A5AtYEAMSJMGqChUfcsqqA)
 Call ID: toolu_bdrk_01A5AtYEAMSJMGqChUfcsqqA
  Args:
    x: 3
    y: 2
================================= Tool Message =================================
Name: find_product

6
================================== Ai Message ==================================

Alright, so 3 multiplied by 2 is 6.

Now let's calculate 5 + 2:
Tool Calls:
  find_sum (toolu_bdrk_013VastrqA6oVF5YjzbfC1ke)
 Call ID: toolu_bdrk_013VastrqA6oVF5YjzbfC1ke
  Args:
    x: 5
    y: 2
================================= Tool Message =====================

In [33]:
agent_graph=create_react_agent(
    model=model, 
    state_modifier=system_prompt,
    tools=agent_tools,
    debug=True)

inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('user', 'what is the sum of 2 and 3 ?')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user', 'what is the sum of 2 and 3 ?')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='ca06eaa5-38fe-4e0e-be63-5f86ac7f74cd')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='ca06eaa5-38fe-4e0e-be63-5f86ac7f74cd')],
 'remaining_steps': 24}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [AIMessage(content='', additional_kwargs={'usage': {'prompt_tokens': 497, 'completion_tokens': 70, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 567}, 'stop_reason': 't